# Profile visualization with seaborne 

In [ ]:
from util import *

import numpy as np 
import pandas as pd
import itertools
import altair as alt
alt.data_transformers.disable_max_rows()
alt.renderers.enable('png')
from altair_visualisation import *
from tqdm import tqdm
idx = pd.IndexSlice
%load_ext autoreload
%autoreload 2

In [ ]:
info_df, data_df = read_data(nrows = 100)
# only keep the last year of each profile 
last_of_each_profile = ~data_df.index.get_level_values(0).duplicated(keep = 'last')
data_df = data_df.loc[last_of_each_profile]
labels = pd.read_csv('full_clustering.csv', index_col = 0)
data_df.head()

In [ ]:
meterID = 'smartmeter_0'
def energy_heatmap_chart(meterID, data_df): 
    subset = (
        data_df.loc[[meterID],:]
        .droplevel(level = 1, axis =0)
        .stack().to_frame('value')
        .reset_index()
        .assign(
            time = lambda x: add_date(x.timestamp.dt.time),
            date = lambda x: add_time(x.timestamp.dt.date),
        )
    )
    return alt.Chart(subset, height = 1000, width = 1000).mark_rect(strokeOpacity = 0).encode(
        x = alt.X('time:O', axis = alt.Axis(labels = False, grid = False)),
        y = alt.Y('date:O', axis = alt.Axis(labels = False, grid = False)), 
        color = alt.Color('value:Q', scale = alt.Scale(scheme = 'viridis'))
    )


In [ ]:
meterID = 'smartmeter_0'
def all_day_plot(meterID, data_df): 
    subset = (
            data_df.loc[[meterID],:]
            .droplevel(level = 1, axis =0)
            .stack().to_frame('value')
            .reset_index()
            .assign(
                time = lambda x: add_date(x.timestamp.dt.time),
                date = lambda x: x.timestamp.dt.date.astype('str'),
            )
        )
    return alt.Chart(subset).mark_line(strokeWidth = 0.2, strokeOpacity = 0.2).encode(
        x = alt.X('time:T', axis = alt.Axis(format = '%H:%M')),
        y = alt.Y('value:Q'), 
        color = alt.Color('date', legend = None, scale = alt.Scale(scheme = 'rainbow'))
    )

In [ ]:
show_clustering(data_df, labels, type ='heatmap')

In [ ]:
show_clustering(data_df, labels)